In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

!pip install tensorflow==2.0.0-alpha0

import numpy as np
import os
import tensorflow as tf
import time



DATA_URL = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
BATCH_SIZE = 512
MAX_LEN = 256
NUM_REVIEWS = 25000


def get_model(input_dim, embedding_dim=50, hidden_units=[100]):
  """Create a Keras Sequential model with layers.

  Args:
    input_dim: (int) Input dimensions for input layer.
    embedding_dim: (int) Embedding dimension for embedding layer.
    hidden_units: [int] the layer sizes of the DNN (input layer first)

  Returns:
    A Keras model.
  """

  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim=input_dim,
                                      output_dim=embedding_dim,
                                      input_length=MAX_LEN))
  model.add(tf.keras.layers.GlobalMaxPool1D())
  for units in hidden_units:
    model.add(tf.keras.layers.Dense(units, activation=tf.keras.backend.relu))
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model


def get_labeled_dataset(patterns):

  # Maps a filename to a dataset that produces (review, sentiment) pair.
  def flat_map_fn(filename):
    label = tf.data.Dataset.from_tensors(
        tf.cast(tf.strings.regex_full_match(filename, '^.*pos.*$'), tf.float64))
    return tf.data.Dataset.zip((tf.data.TextLineDataset(filename), label))

  files = tf.data.Dataset.list_files(patterns).shuffle(NUM_REVIEWS)
  return files.flat_map(flat_map_fn)


def tokenize(line, label):
  # Replace line breaks with spaces.
  line = tf.strings.regex_replace(line, r'\<br \/\>', ' ')
  # Replace periods with spaces.
  line = tf.strings.regex_replace(line, r'\.', ' ')
  tokens = tf.strings.split([line], sep=" ").values
  return tokens, label


def get_vocabulary(patterns):
  dataset = get_labeled_dataset(patterns)
  dataset = dataset.map(tokenize)
  dataset = dataset.flat_map(lambda x, y: tf.data.Dataset.from_tensor_slices(x))
  dataset = dataset.apply(tf.data.experimental.unique())
  dataset = tf.data.Dataset.zip((dataset, tf.data.experimental.Counter()))

  vocabulary = {}
  for word, index in iter(dataset):
    vocabulary[word.numpy()] = index
  return vocabulary


def get_indexed_dataset(vocabulary, patterns):

  def index_and_pad(word_list, label):

    def helper(word_list):
      result = []
      for word, _ in zip(word_list, range(MAX_LEN)):
        result.append(vocabulary[word])
      return tf.pad(result, [[0, MAX_LEN - len(result)]], 'CONSTANT')

    return tf.numpy_function(helper, [word_list], tf.int64), label

  dataset = get_labeled_dataset(patterns)
  dataset = dataset.map(tokenize)
  dataset = dataset.map(index_and_pad)
  dataset = dataset.shuffle(10 * BATCH_SIZE)
  dataset = dataset.batch(BATCH_SIZE)
  return dataset


file = 'aclImdb_v1.tar.gz'
file = tf.keras.utils.get_file(file, DATA_URL, extract=True)
path = os.path.join(os.path.dirname(file), 'aclImdb')

train_files = [
    os.path.join(path, 'train', label, '*') for label in ['pos', 'neg']
]
test_files = [
    os.path.join(path, 'test', label, '*') for label in ['pos', 'neg']
]

vocabulary = get_vocabulary(train_files + test_files)
print(len(vocabulary))
train_data = get_indexed_dataset(vocabulary, train_files)

model = get_model(len(vocabulary))
model.fit(train_data, epochs=10)

test_data = get_indexed_dataset(vocabulary, test_files)
model.evaluate(test_data)


345418


W0321 22:29:00.914769 140374431422336 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/script_ops.py:476: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


Epoch 1/10
49/49 [==============================] - 38s 769ms/step - loss: 0.6856 - accuracy: 0.5904
Epoch 2/10
49/49 [==============================] - 36s 744ms/step - loss: 0.5781 - accuracy: 0.8380
Epoch 3/10
49/49 [==============================] - 37s 749ms/step - loss: 0.3396 - accuracy: 0.8825
Epoch 4/10
49/49 [==============================] - 37s 750ms/step - loss: 0.1999 - accuracy: 0.9308
Epoch 5/10
49/49 [==============================] - 39s 800ms/step - loss: 0.1140 - accuracy: 0.9659
Epoch 6/10
49/49 [==============================] - 38s 771ms/step - loss: 0.0593 - accuracy: 0.9877
Epoch 7/10
49/49 [==============================] - 37s 765ms/step - loss: 0.0296 - accuracy: 0.9963
Epoch 8/10
49/49 [==============================] - 37s 757ms/step - loss: 0.0152 - accuracy: 0.9990
Epoch 9/10
49/49 [==============================] - 36s 743ms/step - loss: 0.0085 - accuracy: 0.9998
Epoch 10/10
49/49 [==============================] - 36s 742ms/step - loss: 0.0053 - accura

[0.37087987332927935, 0.85696]